Code to warm up kernel

In [1]:
#Imports
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from pytrends.request import TrendReq
#Declare constants
ETH_TICKER = "ETH-USD"
SPX_TICKER = "^GSPC" 
BTC_TICKER = "BTC-USD"

In [2]:
print("hello world")

hello world


Below is the code to get data for:

Ether Vs S&P 500 data!

&

BTC Vs S&P 500 data!

In [3]:
# --- 1. Data Retrieval and DataFrame Creation ---
START_DATE = "2020-01-01"

try:
    # --- FETCH ETH DATA ---
    eth_data = yf.Ticker(ETH_TICKER).history(start=START_DATE, period="5y")
    df_eth = eth_data[['Open', 'High', 'Low', 'Close','Volume']].copy()

    # --- FETCH S&P 500 DATA ---
    spx_data = yf.Ticker(SPX_TICKER).history(start=START_DATE, period="5y")
    df_spx = spx_data[['Close']].copy()
    df_spx.rename(columns={'Close': 'SPX_Close'}, inplace=True)


    # --- 2. Data Processing: Resample, Align, and Join ---
    
    # Resample ETH daily data to MONTHLY averages
    df_eth_monthly = df_eth.resample('M').mean()
    
    # Resample S&P 500 daily data to MONTHLY averages
    df_spx_monthly = df_spx.resample('BM').mean()

    # CRITICAL FIX: Convert both Datetime Indexes to Month Period Indexes for alignment
    df_eth_monthly.index = df_eth_monthly.index.to_period('M')
    df_spx_monthly.index = df_spx_monthly.index.to_period('M')

    # Join the two monthly DataFrames on their common Period Index
    df_combined_monthly_eth = df_eth_monthly.join(
        df_spx_monthly, 
        how='inner'
    )
    
    # Convert the PeriodIndex back to a DatetimeIndex for plotting
    df_combined_monthly_eth.index = df_combined_monthly_eth.index.to_timestamp()

    # --- Summary of Combined Table ---
    print(f"--- Combined Monthly Average Data (ETH-USD and S&P 500) ---")
    print(df_combined_monthly_eth.head().to_markdown())
    print("Tail. . .")
    print(df_combined_monthly_eth.tail().to_markdown())
    print("END OF TABLE")

except Exception as e:
    print(f"An error occurred while fetching data: {e}")
    print("Please ensure you have the 'yfinance' library installed: pip install yfinance")
    exit()

--- Combined Monthly Average Data (ETH-USD and S&P 500) ---
| Date                |    Open |    High |     Low |   Close |      Volume |   SPX_Close |
|:--------------------|--------:|--------:|--------:|--------:|------------:|------------:|
| 2020-01-01 00:00:00 | 155.32  | 159.443 | 152.113 | 156.973 | 1.05799e+10 |     3278.2  |
| 2020-02-01 00:00:00 | 237.377 | 245.454 | 229.152 | 238.756 | 1.99342e+10 |     3277.31 |
| 2020-03-01 00:00:00 | 163.235 | 169.512 | 154.013 | 160.376 | 1.60942e+10 |     2652.39 |
| 2020-04-01 00:00:00 | 169.488 | 175.957 | 166.019 | 171.947 | 1.77386e+10 |     2761.98 |
| 2020-05-01 00:00:00 | 207.347 | 212.502 | 202.587 | 208.126 | 1.63095e+10 |     2919.61 |
Tail. . .
| Date                |    Open |    High |     Low |   Close |      Volume |   SPX_Close |
|:--------------------|--------:|--------:|--------:|--------:|------------:|------------:|
| 2024-08-01 00:00:00 | 2665.17 | 2729.73 | 2570.7  | 2642.02 | 1.72829e+10 |     5478.21 |
| 2024-09-

C:\Users\Kevin Ivander\AppData\Local\Temp\ipykernel_10960\1830324824.py:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_eth_monthly = df_eth.resample('M').mean()
C:\Users\Kevin Ivander\AppData\Local\Temp\ipykernel_10960\1830324824.py:21: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  df_spx_monthly = df_spx.resample('BM').mean()
C:\Users\Kevin Ivander\AppData\Local\Temp\ipykernel_10960\1830324824.py:24: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_eth_monthly.index = df_eth_monthly.index.to_period('M')
C:\Users\Kevin Ivander\AppData\Local\Temp\ipykernel_10960\1830324824.py:25: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_spx_monthly.index = df_spx_monthly.index.to_period('M')


In [4]:
# --- 1. Data Retrieval and DataFrame Creation ---

# Define the ticker symbols
START_DATE = "2020-01-01"

try:
    # --- FETCH ETH DATA ---
    btc_data = yf.Ticker(BTC_TICKER).history(start=START_DATE, period="5y")
    df_btc = btc_data[['Open', 'High', 'Low', 'Close','Volume']].copy()

    # --- FETCH S&P 500 DATA ---
    spx_data = yf.Ticker(SPX_TICKER).history(start=START_DATE, period="5y")
    df_spx = spx_data[['Close']].copy()
    df_spx.rename(columns={'Close': 'SPX_Close'}, inplace=True)


    # --- 2. Data Processing: Resample, Align, and Join ---
    
    # Resample BTC daily data to MONTHLY averages
    df_btc_monthly = df_btc.resample('MS').mean()
    
    # Resample S&P 500 daily data to MONTHLY averages
    df_spx_monthly = df_spx.resample('MS').mean()

    # CRITICAL FIX: Convert both Datetime Indexes to Month Period Indexes for alignment
    df_btc_monthly.index = df_btc_monthly.index.to_period('M')
    df_spx_monthly.index = df_spx_monthly.index.to_period('M')

    # Join the two monthly DataFrames on their common Period Index
    df_combined_monthly_btc = df_btc_monthly.join(
        df_spx_monthly, 
        how='inner'
    )
    
    # Convert the PeriodIndex back to a DatetimeIndex for plotting
    df_combined_monthly_btc.index = df_combined_monthly_btc.index.to_timestamp()

    # --- Summary of Combined Table ---
    print(f"--- Combined Monthly Average Data (BTC-USD and S&P 500) ---")
    print(df_combined_monthly_btc.head().to_markdown())
    print("Tail. . .")
    print(df_combined_monthly_btc.tail().to_markdown())
    print("END OF TABLE")
except Exception as e:
    print(f"An error occurred while fetching data: {e}")
    exit()

--- Combined Monthly Average Data (BTC-USD and S&P 500) ---
| Date                |    Open |    High |     Low |   Close |      Volume |   SPX_Close |
|:--------------------|--------:|--------:|--------:|--------:|------------:|------------:|
| 2020-01-01 00:00:00 | 8318.95 | 8490.32 | 8203    | 8389.27 | 2.7512e+10  |     3278.2  |
| 2020-02-01 00:00:00 | 9656.22 | 9791.16 | 9473.44 | 9630.72 | 4.01164e+10 |     3277.31 |
| 2020-03-01 00:00:00 | 6943.51 | 7182.89 | 6619.61 | 6871.02 | 4.16272e+10 |     2652.39 |
| 2020-04-01 00:00:00 | 7150.61 | 7352.33 | 7045.44 | 7224.48 | 3.85376e+10 |     2761.98 |
| 2020-05-01 00:00:00 | 9237.76 | 9451.19 | 9048.87 | 9263.15 | 4.14957e+10 |     2919.61 |
Tail. . .
| Date                |    Open |     High |     Low |   Close |      Volume |   SPX_Close |
|:--------------------|--------:|---------:|--------:|--------:|------------:|------------:|
| 2024-08-01 00:00:00 | 60103.9 |  61477.3 | 58500.2 | 59921.2 | 3.36486e+10 |     5478.21 |
| 2024-

C:\Users\Kevin Ivander\AppData\Local\Temp\ipykernel_10960\3149198867.py:26: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_btc_monthly.index = df_btc_monthly.index.to_period('M')
C:\Users\Kevin Ivander\AppData\Local\Temp\ipykernel_10960\3149198867.py:27: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_spx_monthly.index = df_spx_monthly.index.to_period('M')


BTC HASHRATE + Fear & Greed index ~

All using dataset

In [5]:
import pandas as pd

FILE_PATH_HASH = "C:\\Users\\Kevin Ivander\\Documents\\GitHub\\DSC101\\csv_files\\hashrateBTC.csv"
START_DATE = '2020-01-01 00:00:00'
END_DATE = '2024-12-31 00:00:00'

df_hashrate_data = pd.read_csv(
    FILE_PATH_HASH,
    sep=';',
    names = ['DateTime', 'Hashrate'],
    index_col='DateTime',
    parse_dates=['DateTime'],
    skiprows=1,
    header=None
    )
df_hashrate = df_hashrate_data.loc[START_DATE:END_DATE].copy()


df_hashrate.index = pd.to_datetime(df_hashrate.index)

df_hashrate['Hashrate'] = pd.to_numeric(df_hashrate['Hashrate'].astype(float))
monthly_avg_hash = df_hashrate.resample('MS').mean()
print("successfully extracted hashrate data")
print(monthly_avg_hash.head())


successfully extracted hashrate data
                Hashrate
DateTime                
2020-01-01  1.092595e+20
2020-02-01  1.104310e+20
2020-03-01  1.069162e+20
2020-04-01  1.115371e+20
2020-05-01  1.057221e+20


In [6]:
# Define parameters
FILE_PATH_FG = r'C:\\Users\\Kevin Ivander\\Documents\\GitHub\\DSC101\\CryptoGreedFear.csv'
START_DATE = '2020-01-01 00:00:00'
END_DATE = '2024-12-31 00:00:00'

# Load the data
df_fng_data = pd.read_csv(FILE_PATH_FG)

# Preprocessing: Convert 'date' to datetime and set as index
df_fng_data['date'] = pd.to_datetime(df_fng_data['date'])
df_fng_data = df_fng_data.set_index('date')

# CRUCIAL: Sort the index for correct date slicing, resolving the previous KeyError
df_fng_data = df_fng_data.sort_index()

# Filter by date range
df_fng = df_fng_data.loc[START_DATE:END_DATE].copy()

#Resample to monthly frequency and calculate mean
monthly_avg = df_fng['fng_value'].resample('MS').mean()

# Convert the Series to a DataFrame for cleaner display
monthly_avg_fng_df = monthly_avg.to_frame(name='Mean FNG Value')
monthly_avg_fng_df.index.name = 'Month Start Date'
# --- Display and Save ---

print("--- Full Monthly Average Table ---")
# Display the full DataFrame
print(monthly_avg_fng_df.to_markdown())

# Save the data to a CSV file for easy access <---------------------------------------
output_file = 'monthly_fng_averages_2021_2024.csv'
monthly_avg_fng_df.to_csv(output_file)
print(f"\nData saved to {output_file}")

--- Full Monthly Average Table ---
| Month Start Date    |   Mean FNG Value |
|:--------------------|-----------------:|
| 2020-01-01 00:00:00 |          53.25   |
| 2020-02-01 00:00:00 |          54.4167 |
| 2020-03-01 00:00:00 |          53.3333 |
| 2020-04-01 00:00:00 |          50.3333 |
| 2020-05-01 00:00:00 |          49.9167 |
| 2020-06-01 00:00:00 |          53.4167 |
| 2020-07-01 00:00:00 |          53.5833 |
| 2020-08-01 00:00:00 |          54.5    |
| 2020-09-01 00:00:00 |          52.6667 |
| 2020-10-01 00:00:00 |          52.0833 |
| 2020-11-01 00:00:00 |          52.0833 |
| 2020-12-01 00:00:00 |          49.4167 |
| 2021-01-01 00:00:00 |          55.4167 |
| 2021-02-01 00:00:00 |          59.4167 |
| 2021-03-01 00:00:00 |          59.0833 |
| 2021-04-01 00:00:00 |          60      |
| 2021-05-01 00:00:00 |          58      |
| 2021-06-01 00:00:00 |          59.9167 |
| 2021-07-01 00:00:00 |          62.8333 |
| 2021-08-01 00:00:00 |          60.6667 |
| 2021-09-01 00:00:

Sychronizing and Scaling

In [7]:
# MIX EVERYTHING INTO ONE DATAFRAME (Seperated for ETH and BTC)
#BTC
df_combined_monthly_btc = df_combined_monthly_btc.join(monthly_avg_fng_df)
df_combined_monthly_btc = df_combined_monthly_btc.join(monthly_avg_hash)
#ETH
df_combined_monthly_eth = df_combined_monthly_eth.join(monthly_avg_fng_df)

In [8]:

# Convert raw HashRate to Exahashes (EH/s) by dividing by 10^18 and volume into bilions
df_combined_monthly_btc['Hashrate'] = df_combined_monthly_btc['Hashrate'] / 1e18
df_combined_monthly_btc['Volume'] = df_combined_monthly_btc['Volume'] / 1e10
df_combined_monthly_eth['Volume'] = df_combined_monthly_eth['Volume'] / 1e10
# Rename the column so you don't forget the unit later
df_combined_monthly_btc.rename(columns={'Hashrate': 'Hashrate_EH'}, inplace=True)
df_combined_monthly_btc.rename(columns={'Volume': 'Volume (Bilions)'}, inplace=True)
df_combined_monthly_eth.rename(columns={'Volume': 'Volume (Bilions)'}, inplace=True)
print(df_combined_monthly_btc.head().to_markdown())
print(df_combined_monthly_eth.head().to_markdown())


| Date                |    Open |    High |     Low |   Close |   Volume (Bilions) |   SPX_Close |   Mean FNG Value |   Hashrate_EH |
|:--------------------|--------:|--------:|--------:|--------:|-------------------:|------------:|-----------------:|--------------:|
| 2020-01-01 00:00:00 | 8318.95 | 8490.32 | 8203    | 8389.27 |            2.7512  |     3278.2  |          53.25   |       109.26  |
| 2020-02-01 00:00:00 | 9656.22 | 9791.16 | 9473.44 | 9630.72 |            4.01164 |     3277.31 |          54.4167 |       110.431 |
| 2020-03-01 00:00:00 | 6943.51 | 7182.89 | 6619.61 | 6871.02 |            4.16272 |     2652.39 |          53.3333 |       106.916 |
| 2020-04-01 00:00:00 | 7150.61 | 7352.33 | 7045.44 | 7224.48 |            3.85376 |     2761.98 |          50.3333 |       111.537 |
| 2020-05-01 00:00:00 | 9237.76 | 9451.19 | 9048.87 | 9263.15 |            4.14957 |     2919.61 |          49.9167 |       105.722 |
| Date                |    Open |    High |     Low |   Close 

In [ ]:
try:
    pytrends = TrendReq(hl='en-US', tz=360, retries=2, backoff_factor=0.5) # hl=host language, tz=timezone offset
    
    # Build the payload (prepare the request)
    pytrends.build_payload(
        kw_list=['Bitcoin', 'Ethereum'], 
        cat=0, 
        timeframe= '2020-01-01 2024-12-31', 
        geo=''
    )
    
    df_trends_daily = pytrends.interest_over_time()
    if 'isPartial' in df_trends_daily.columns:
        df_trends_daily = df_trends_daily.drop(columns=['isPartial'])
    df_trends_daily.index = df_trends_daily.index.normalize()
    df_trends_monthly = df_trends_daily.resample('MS').mean()
    df_trends_monthly.columns = ['GTrends_BTC', 'GTrends_ETH']
    
    print("--- Google Trends Monthly Average Data ---")
    print(df_trends_monthly.head().to_markdown())
    
except Exception as e:
    print(f"An error occurred while fetching Google Trends data: {e}")
    

df_combined_monthly_btc = df_combined_monthly_btc.join(df_trends_monthly['GTrends_BTC'])
df_combined_monthly_eth = df_combined_monthly_eth.join(df_trends_monthly['GTrends_ETH'])

An error occurred while fetching Google Trends data: The request failed: Google returned a response with code 429
